In [39]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm
from FetchLatLon import get_latlon

# Fetch data from gov
url = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vQjf_HNeEZKM-XJX-q5v4cfNrB3kcv4gOT8kFbV9rurfoX_H5Qv9112Pv0PgYNFSzbReyNlQkLrJib3/pubhtml?utm_source=google&utm_medium=cpc&utm_campaign=%E6%B8%85%E5%86%A0%E4%B8%80%E8%99%9F%E5%8B%95%E6%85%8B%E8%A1%A8#'
resp = requests.get(url).text
soup = BeautifulSoup(resp, 'html.parser')

# parse data from html table to DF
DF = pd.read_html(str(soup.table), encoding='utf-8')[0]
DF.columns = pd.MultiIndex.from_arrays(DF.iloc[0:2].values)
DF = DF.iloc[3:, 1:]
DF = DF.set_index((None, '編號'))

In [40]:
DF[('LatLon', 'LatLon')] = None
DF

最新用藥指引 全聯會提供公費清冠一號民眾意見反應專區  順天堂  莊松榮   立康   勝昌  勸奉堂   華陀   天一   漢聖  \
              縣市別              醫療院所名稱 剩餘人次 剩餘人次 剩餘人次 剩餘人次 剩餘人次 剩餘人次 剩餘人次 剩餘人次   
(None, 編號)                                                                      
798           南投縣              隆安中醫診所    0    0    0    0    0   93    0    0   
826           南投縣              永安中醫診所    0   80    0    0    0    5    0    0   
1506          南投縣            中興大愛中醫診所    0    0    0    0    0    0    8    0   
795           南投縣              德安中醫診所    0    0    0    0    0  4.5    0    0   
816           南投縣              首茗中醫診所    0    0    0    0   10    0    0    2   
...           ...                 ...  ...  ...  ...  ...  ...  ...  ...  ...   
1509          高雄市              祐祥中醫診所    0    0    0    0    0    0    0    0   
1573          高雄市     義大醫療財團法人義大癌治療醫院    0    0    0    0    0    0    0    0   
1588          高雄市              五乙中醫診所    0    0    0    0    0    0    0    0   
1589          高雄市              祥霖中醫診所    0    0    0    0    0    0    0    0   
1602          高雄市              濟民中醫診所    0    0    0    0    0    0    0    0   

            ...   富田   2022/8/17 下午 1:33:03 原廠清冠一號照片 清冠一號Q&A收錄集     NaN  \
            ... 剩餘人次             醫療院所最新更新時間   清冠一號公費     清冠一號自費 星期日特約門診   
(None, 編號)  ...                                                           
798         ...    0   2022/8/17 下午 1:10:35        是          是       是   
826         ...    0  2022/8/17 上午 11:44:47        是        NaN     NaN   
1506        ...    0  2022/8/17 上午 11:24:44        是          是     NaN   
795         ...    0  2022/8/17 上午 10:42:09        是        NaN     NaN   
816         ...    0  2022/8/16 下午 10:50:25        是          是     NaN   
...         ...  ...                    ...      ...        ...     ...   
1509        ...    0  2022/5/20 上午 12:00:00      NaN        NaN     NaN   
1573        ...    0  2022/5/20 上午 12:00:00      NaN        NaN     NaN   
1588        ...    0  2022/5/20 上午 12:00:00      NaN        NaN     NaN   
1589        ...    0  2022/5/20 上午 12:00:00      NaN        NaN     NaN   
1602        ...    0  2022/5/20 上午 12:00:00      NaN        NaN     NaN   

                   就診中醫院所前請先點選西醫快篩陽性判斷視訊院所               NaN                   \
           診療長新冠症狀          下方為中醫院所LINE ID                電話               地址   
(None, 編號)                                                                      
798              是       long-an0492222316       049-2222317       南投市彰南路2段8號   
826              是              0492323166       049-2323166   南投縣草屯鎮明賢街一段28號   
1506             是                   請來電洽詢       049-2331339      南投縣南投市學藝路5號   
795              是                     NaN       049-2207649     南投市彰南路2段510號   
816              是              0492985015       049-2985015     埔里鎮東華路120號1F   
...            ...                     ...               ...              ...   
1509           NaN                     NaN        07-3657888  高雄市楠梓區右昌街718號1樓   
1573           NaN                     NaN  07-6150022分機6112     高雄市燕巢區義大路21號   
1588           NaN                     NaN      (07) 3526051     高雄市楠梓區興楠路17號   
1589           NaN                     NaN      (07) 3946788    高雄市三民區聯興路125號   
1602           NaN                     NaN        07-2410822    高雄市新興區中山一路61號   

           LatLon  
           LatLon  
(None, 編號)         
798          None  
826          None  
1506         None  
795          None  
816          None  
...           ...  
1509         None  
1573         None  
1588         None  
1589         None  
1602         None  

[1555 rows x 22 columns]

In [41]:
for i, row in tqdm(DF.iterrows(), total=DF.shape[0]):
    row = list(row)
    DF.loc[i, [('LatLon', 'LatLon')]] = get_latlon(row[-2])
DF

['DOCTYPE html PUBLIC ']


  2%|▏         | 25/1555 [00:10<10:39,  2.39it/s]

['DOCTYPE html PUBLIC ']


  2%|▏         | 26/1555 [00:11<11:11,  2.28it/s]

['DOCTYPE html PUBLIC ']


...

['DOCTYPE html PUBLIC ']


  2%|▏         | 38/1555 [00:15<08:31,  2.97it/s]

['DOCTYPE html PUBLIC ']


  2%|▏         | 38/1555 [00:15<10:32,  2.40it/s]


KeyboardInterrupt: 